In [1]:
# Check the lammps is working
import lammps
lmp = lammps.lammps()
print(lmp)


LAMMPS (15 Jun 2023 - Development - patch_15Jun2023-196-ge25cbde6e2-modified)


In [2]:
import numpy as np
from fitsnap3lib.fitsnap import FitSnap

# Step 1: Train the Ensemble of NNs
## Define the setting dictionary for training the NNs

In [3]:
# Create an input dictionary containing settings.

settings = \
{
"BISPECTRUM":
    {
    "numTypes": 1,
    "twojmax": 8,
    "rcutfac": 4.1,
    "rfac0": 0.99363,
    "rmin0": 0.0,
    "wj": 1.0,
    "radelem": 0.5,
    "type": "Ni",
    "wselfallflag": 0,
    "chemflag": 0,
    "bzeroflag": 0,
    "quadraticflag": 0,
     "bikflag": 1,
    "dgradflag": 1,
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSSNAP",
    "energy": 1,
    "per_atom_energy": 1,
    "force": 1,
    "stress": 0,
        "nonlinear": 1
    },
"ESHIFT":
    {
    "Ni": 0.0
    },
"PYTORCH":
    {
    "layer_sizes": "num_desc 55 64 32 16 1",
    "learning_rate": 1.5e-4,
    "num_epochs": 10,
    "batch_size": 4, # 363 configs in entire set
    "save_state_output": "Ni_Pytorch.pt",
     "energy_weight": 1.0,
        "force_weight": 10.0,
        "training_fraction": 1
    },
"SOLVER":
    {
    "solver": "PYTORCH",
    "compute_testerrs": 1,
    "detailed_errors": 1
    },
"SCRAPER":
    {
    "scraper": "JSON" 
    },
"PATH":
    {
    "dataPath": "Initial_training_set"
    },
"OUTFILE":
    {
    "metrics": "Ni_metrics.md",
    "potential": "Ni_pot"
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "zero 10.0",
    "pair_coeff1": "* *",
    },
"EXTRAS":
    {
    "dump_peratom": 1,
    "dump_perconfig": 1
    },
"GROUPS":
    {
    "group_sections": "name training_size testing_size eweight fweight vweight",
    "group_types": "str float float float float float",
    "smartweights": 0,
    "random_sampling": 0,
    "Ni_JSON" :  "1  0  1.0e6 1.0e6 1.0e-12"
    }
}

In [4]:
#start unstance of fitsnap
fs = FitSnap(settings, arglist=["--overwrite"])

----- Global weights set: Overriding group weights.
----- Global training fraction set: Overriding group fractions.


In [5]:
# scrape data
fs.scrape_configs()

'scrape_configs' took 404.78 ms on rank 0


In [6]:
print(len(fs.data))

294


In [7]:
# print out example config
print(fs.data[0])

{'AtomTypeStyle': 'chemicalsymbol', 'EnergyStyle': 'electronvolt', 'ForcesStyle': 'electronvoltperangstrom', 'LatticeStyle': 'angstrom', 'PositionsStyle': 'angstrom', 'StressStyle': 'kB', 'File': 'Mo_10_vasp_1.json', 'Group': 'Ni_JSON', 'AtomTypes': ['Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni'], 'Energy': -554.67899589, 'Forces': array([[ 1.042700e-02, -8.223610e-01, -6.843620e-01],
       [ 1.353150

In [8]:
# calculate the descriptors
fs.process_configs()

/Users/adrohsk/miniconda3/envs/fitsnap/lib/python3.10/site-packages/lammps/mliap/__init__.py:30: UserWarning: This interpreter is not compatible with python-based MLIAP for LAMMPS. Attempting to activate the MLIAP-python coupling from python may result in undefined behavior.
  warnings.warn("This interpreter is not compatible with python-based MLIAP for LAMMPS. "


'process_configs' took 41099.92 ms on rank 0


In [9]:
# perform fit
fs.perform_fit()

Epoch   Train       Val     Time (s)


0  2.013e+01     nan     2.688e+00


/Users/adrohsk/miniconda3/envs/fitsnap/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/adrohsk/miniconda3/envs/fitsnap/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1  1.327e+00     nan     2.595e+00
2  7.202e-01     nan     2.818e+00
3  5.276e-01     nan     2.941e+00
4  4.247e-01     nan     2.776e+00
5  3.533e-01     nan     2.666e+00
6  3.018e-01     nan     2.675e+00
7  2.534e-01     nan     2.809e+00
8  2.219e-01     nan     2.790e+00
9  1.921e-01     nan     2.820e+00
'fit' took 27671.11 ms on rank 0
'error_analysis' took 2500.53 ms on rank 0


In [10]:
# Write LAMMPS potential files.
fs.output.write_lammps(fs.solver.fit)
# Look at files:
!ls -rtl

#why is it not saving it as Ni_Pytorch?

total 11064
drwxr-xr-x   3 adrohsk  SANDIA\Domain Users       96 Aug 16 09:21 Initial_training_set
drwxr-xr-x   3 adrohsk  SANDIA\Domain Users       96 Aug 16 09:21 Unlabeled_set
drwxr-xr-x   4 adrohsk  SANDIA\Domain Users      128 Aug 16 09:40 config_file_Uset
-rw-r--r--   1 adrohsk  SANDIA\Domain Users    49360 Aug 16 09:40 Active_learning_pipeline.ipynb
drwxr-xr-x  10 adrohsk  SANDIA\Domain Users      320 Aug 16 09:43 input_script_2_fit_nn
-rw-r--r--   1 adrohsk  SANDIA\Domain Users       78 Aug 16 11:18 log.lammps
-rw-r--r--   1 adrohsk  SANDIA\Domain Users   162475 Aug 16 11:19 Ni_Pytorch.pt
-rw-r--r--   1 adrohsk  SANDIA\Domain Users      540 Aug 16 11:19 loss_vs_epochs.dat
-rw-r--r--   1 adrohsk  SANDIA\Domain Users   105919 Aug 16 11:19 FitTorch_Pytorch.pt
-rw-r--r--   1 adrohsk  SANDIA\Domain Users    21953 Aug 16 11:19 perconfig.dat
-rw-r--r--   1 adrohsk  SANDIA\Domain Users  4451086 Aug 16 11:19 peratom.dat


# Now make loop to automate

In [11]:
# make a list of instances to loop around


In [12]:
# loop over all instansces and fit
# create N fitsnap instances to create N different .pt files

#move pt file to a new folder call fits and have the following structure AL_#/NN_#/Ni_pytorch.pt

# Step 2 

In [13]:
import sys
import numpy as np
import pickle
import torch
from pathlib import Path
import glob
# from fitsnap3lib.fitsnap import FitSnap
import pickle
import time

list_of_weights=sorted(glob.glob('Ni*.pt'))
print('These are the weights of the nn',list_of_weights)


These are the weights of the nn ['Ni_Pytorch.pt']


In [15]:
# load in the config file 

print('Loading config file')

t0 = time.time()

with open(r"config_file_Uset/configs.pickle", "rb") as file:
    configs = pickle.load(file)

t1 = time.time()
total = t1-t0

print('Total time (s) to load config file',total)

Loading config file
Total time (s) to load config file 1.3967540264129639


In [16]:
# Create an input dictionary containing settings.

settings_eval = \
{
"BISPECTRUM":
    {
    "numTypes": 1,
    "twojmax": 8,
    "rcutfac": 4.1,
    "rfac0": 0.99363,
    "rmin0": 0.0,
    "wj": 1.0,
    "radelem": 0.5,
    "type": "Ni",
    "wselfallflag": 0,
    "chemflag": 0,
    "bzeroflag": 0,
    "quadraticflag": 0,
     "bikflag": 1,
    "dgradflag": 1,
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSSNAP",
    "energy": 1,
    "per_atom_energy": 1,
    "force": 1,
    "stress": 0,
        "nonlinear": 1
    },
"ESHIFT":
    {
    "Ni": 0.0
    },
"PYTORCH":
    {
    "layer_sizes": "num_desc 55 64 32 16 1",
    "learning_rate": 1.5e-4,
    "num_epochs": 10,
    "batch_size": 4, # 363 configs in entire set
    "save_state_input": "Ni_Pytorch.pt",
     "energy_weight": 1.0,
        "force_weight": 10.0,
        "training_fraction": 1
    },
"SOLVER":
    {
    "solver": "PYTORCH",
    "compute_testerrs": 1,
    "detailed_errors": 1
    },
"SCRAPER":
    {
    "scraper": "JSON" 
    },
"PATH":
    {
    "dataPath": "/home/localdmonte/fs-ensembles/AL_vasp_pipeline/Initial_training_set"
    },
"OUTFILE":
    {
    "metrics": "Ni_metrics.md",
    "potential": "Ni_pot"
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "zero 10.0",
    "pair_coeff1": "* *",
    },
"EXTRAS":
    {
    "dump_peratom": 1,
    "dump_perconfig": 1
    },
"GROUPS":
    {
    "group_sections": "name training_size testing_size eweight fweight vweight",
    "group_types": "str float float float float float",
    "smartweights": 0,
    "random_sampling": 0,
    "Ni_JSON" :  "1  0  1.0e6 1.0e6 1.0e-12"
    }
}

In [17]:
# Make a list of settings for each fit.
# Declare number of fits.
nfits = len(list_of_weights)

from copy import deepcopy
settings_lst = [deepcopy(settings_eval) for _ in range(nfits)]
for i,s in enumerate(settings_lst):
    s['PYTORCH']['save_state_input'] = list_of_weights[i]
print(settings_lst)

[{'BISPECTRUM': {'numTypes': 1, 'twojmax': 8, 'rcutfac': 4.1, 'rfac0': 0.99363, 'rmin0': 0.0, 'wj': 1.0, 'radelem': 0.5, 'type': 'Ni', 'wselfallflag': 0, 'chemflag': 0, 'bzeroflag': 0, 'quadraticflag': 0, 'bikflag': 1, 'dgradflag': 1}, 'CALCULATOR': {'calculator': 'LAMMPSSNAP', 'energy': 1, 'per_atom_energy': 1, 'force': 1, 'stress': 0, 'nonlinear': 1}, 'ESHIFT': {'Ni': 0.0}, 'PYTORCH': {'layer_sizes': 'num_desc 55 64 32 16 1', 'learning_rate': 0.00015, 'num_epochs': 10, 'batch_size': 4, 'save_state_input': 'Ni_Pytorch.pt', 'energy_weight': 1.0, 'force_weight': 10.0, 'training_fraction': 1}, 'SOLVER': {'solver': 'PYTORCH', 'compute_testerrs': 1, 'detailed_errors': 1}, 'SCRAPER': {'scraper': 'JSON'}, 'PATH': {'dataPath': '/home/localdmonte/fs-ensembles/AL_vasp_pipeline/Initial_training_set'}, 'OUTFILE': {'metrics': 'Ni_metrics.md', 'potential': 'Ni_pot'}, 'REFERENCE': {'units': 'metal', 'atom_style': 'atomic', 'pair_style': 'zero 10.0', 'pair_coeff1': '* *'}, 'EXTRAS': {'dump_peratom': 

In [18]:

# Load pytorch file from a previous fit.
instances = [FitSnap(settings, arglist=["--overwrite"]) for _ in range(nfits)]
for i, inst in enumerate(instances):
    t0 = time.time()
    inst.solver.configs = configs
    (energies_model, forces_model) = inst.solver.evaluate_configs(config_idx=None, standardize_bool=False)
    t1 = time.time()
    total = t1-t0
    print('Total time (s) to obtain forces and energies from config file',total)

    with open("energies_model_"+str(i+1).zfill(2), "wb") as fp:   #Pickling
        pickle.dump(energies_model, fp)
        
    with open("forces_model_"+str(i+1).zfill(2), "wb") as fp:   #Pickling
        pickle.dump(forces_model, fp)

    # Delete the instance to free memory.
    del inst

----- Global weights set: Overriding group weights.
----- Global training fraction set: Overriding group fractions.
Total time (s) to obtain forces and energies from config file 4.063306093215942


# Step 3 

In [18]:
len(configs)

configs

print(configs[0].filename)
print(configs[0].natoms)
print(configs[0].descriptors.shape)

from tqdm import tqdm
name_list_energy=[]

for i in tqdm(range(len(configs))):
    name_list_energy.append(configs[i].filename)

print(len(name_list_energy))

name_list_forces=[]

for i in tqdm(range(len(configs))):
    for j in range(configs[i].natoms*3):
        
        name_list_forces.append(configs[i].filename)

print(len(name_list_forces))

Ni_uspex_32_6.json
32
(32, 55)


100%|█████████████████████████████████████████████████████████████████████████████████| 8005/8005 [00:00<00:00, 2282953.93it/s]


8005


100%|██████████████████████████████████████████████████████████████████████████████████| 8005/8005 [00:00<00:00, 107451.26it/s]

768480


In [ ]:
# previous energies
previous_energies_1=np.load('file_with_previously_performed_names')
print(previous_energies_1[:10])
print(np.unique(previous_energies_1).shape)
previous_energies_1.shape

# previous forces

previous_forces_1=np.load('file_with_previously_performed_names')
print(previous_forces_1[:10])
print(np.unique(previous_forces_1).shape)
previous_forces_1.shape


In [ ]:
# load all the dump files

with open("energies_model_"+str(1).zfill(2), "rb") as fp:   # Unpickling
    
    NN_1 = pickle.load(fp)
 

In [ ]:
# transform into np array
NN_1_energy =torch.stack(NN_1).detach().numpy()
NN_2_energy =torch.stack(NN_2).detach().numpy()
NN_3_energy =torch.stack(NN_3).detach().numpy()
NN_4_energy =torch.stack(NN_4).detach().numpy()
NN_5_energy =torch.stack(NN_5).detach().numpy()

In [ ]:
# make one array
all_energy=np.array([NN_1_energy,NN_2_energy,NN_3_energy,NN_4_energy,NN_5_energy])
all_energy.shape

In [ ]:
# remove all previous 
all_energy=all_energy[:,ndx_2_keep]
all_energy.shape

In [ ]:
# now for forces

with open("forces_model_"+str(1).zfill(2), "rb") as fp:   # Unpickling
    
    NN_1_force = pickle.load(fp)
    
with open("forces_model_"+str(2).zfill(2), "rb") as fp:   # Unpickling
    
    NN_2_force = pickle.load(fp)
    
with open("forces_model_"+str(3).zfill(2), "rb") as fp:   # Unpickling
    
    NN_3_force = pickle.load(fp)
    
with open("forces_model_"+str(4).zfill(2), "rb") as fp:   # Unpickling
    
    NN_4_force = pickle.load(fp)
    
with open("forces_model_"+str(5).zfill(2), "rb") as fp:   # Unpickling
    
    NN_5_force = pickle.load(fp)

In [ ]:
NN_1_force =torch.cat(NN_1_force).detach().numpy()
NN_2_force =torch.cat(NN_2_force).detach().numpy()
NN_3_force =torch.cat(NN_3_force).detach().numpy()
NN_4_force =torch.cat(NN_4_force).detach().numpy()
NN_5_force =torch.cat(NN_5_force).detach().numpy()

In [ ]:
all_forces=np.array([NN_1_force,NN_2_force,NN_3_force,NN_4_force,NN_5_force])
all_forces.shape

# Step 4

In [ ]:
std_all=all_energy.std(axis=0)
std_all.shape

In [ ]:
# uncertain energies

percentile_of_interest=np.percentile(std_all,99.99)
print(percentile_of_interest)

index_all=np.arange(len(std_all))

index_of_interest=index_all[std_all>percentile_of_interest]
print(len(index_of_interest))

energy_uncertain_configs=np.array(name_list_energy_AL)[index_of_interest]

# certain forces

percentile_of_interest_certain=np.percentile(std_all,.01)
print(percentile_of_interest_certain)

index_all_certain=np.arange(len(std_all))
print(len(index_all_certain))

index_of_interest_certain=index_all_certain[std_all<percentile_of_interest_certain]
print(len(index_of_interest_certain))

energy_certain_configs=np.array(name_list_energy_AL)[index_of_interest_certain]

In [ ]:
std_all_forces=all_forces.std(axis=0)
std_all_forces.shape

In [ ]:
# uncertain forces
percentile_of_interest_forces=np.percentile(std_all_forces,99.9999)
print(percentile_of_interest_forces)

index_all_forces=np.arange(len(std_all_forces))

index_of_interest_forces=index_all_forces[std_all_forces>percentile_of_interest_forces]
print(len(index_of_interest_forces))

print(std_all_forces[index_of_interest_forces[0]])

print(all_forces[:,index_of_interest_forces[0]])

forces_uncertain_configs=np.unique(np.array(name_list_forces_AL)[index_of_interest_forces])
print('Uncertain Forces',forces_uncertain_configs.shape)

# certain forces

percentile_of_interest_certain_forces=np.percentile(std_all_forces,.0001)
print(percentile_of_interest_certain_forces)

index_all_certain_forces=np.arange(len(std_all_forces))

index_of_interest_certain_forces=index_all_certain_forces[std_all_forces<percentile_of_interest_certain_forces]
print(len(index_of_interest_certain_forces))

forces_certain_configs=np.unique(np.array(name_list_forces_AL)[index_of_interest_certain_forces])
print('Certain Forces',forces_certain_configs.shape)

# Step 5


# Steps

1) Train 5 NN potentials using Fitsnap Library. TODO: Save `Ni_Pytorch.pt` files

2) Evaluate the 5 NN potentials using Fitsnap Library

3) Remove from evaluated results the configurations you already performed

4) Select the N most certain and uncertain

5) Run them using VASP. TODO Drew.

6) Repeat